# **Latihan Data Exploration NLP**

NLP adalah cabang dari Artificial Intelligence yang berhubungan dengan interaksi antara mesin dan manusia menggunakan bahasa natural.

Eksplorasi data merupakan langkah untuk memahami data sebelum dilakukan praproses. Pemahaman terhadap data yang akan di-mining dapat membantu dalam menentukan teknik-teknik pra-proses dan analisis data terhadap data sebelum dilakukan data mining.

## **Import Library**

In [ ]:
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

!pip install sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from wordcloud import WordCloud

Import data
silahkan gunakan data yang sudah kalian ambil pada hari selasa, data berupa text yang diambil di twitter dengan topik tertentu

atau bisa gunakan data yang disediakan
[disini](https://drive.google.com/file/d/10sIjKMFBhBlp5W4zCWFwXDars2c7koyR/view?usp=sharing)


In [ ]:
tweet = pd.read_csv('/content/bjorka.csv')
tweet

In [ ]:
tweets = tweet.iloc[0:300,:]
tweets

##Fungsi Preprocessing
Text preprocessing adalah suatu proses untuk menyeleksi data text agar menjadi lebih terstruktur lagi dengan melalui serangkaian tahapan yang meliputi tahapan case folding, tokenizing, filtering dan stemming. Tapi, sesungguhnya tidak ada aturan pasti tentang setiap tahapan dalam text preprocessing. Semua itu tergantung dengan jenis serta kondisi data yang kita miliki. Text preprocessing merupakan salah satu implementasi dari text mining. Text mining sendiri adalah suatu kegiatan menambang data, dimana data yang biasanya diambil berupa text yang bersumber dari dokumen-dokumen yang memiliki goals untuk mencari kata kunci yang mewakili dari sekumpulan dokumen tersebut sehingga nantinya dapat dilakukan analisa hubungan antara dokumen-dokumen tersebut.

In [ ]:
def cleaningText(text):
    text = re.sub(r'@[A-Za-z0-9]+', '', text) # remove mentions
    text = re.sub(r'#[A-Za-z0-9]+', '', text) # remove hashtag
    text = re.sub(r'RT[\s]', '', text) # remove RT
    text = re.sub(r"http\S+", '', text) # remove link
    text = re.sub(r'[0-9]+', '', text) # remove numbers
    text = re.sub(r'B|b', '', text)
    text = re.sub(r'xexxa', '', text)

    text = text.replace('\n', ' ') # replace new line into space
    text = text.translate(str.maketrans('', '', string.punctuation)) # remove all punctuations
    text = text.strip(' ') # remove characters space from both left and right text
    return text

def casefoldingText(text): # Converting all the characters in a text into lower case
    text = text.lower()
    return text

def tokenizingText(text): # Tokenizing or splitting a string, text into a list of tokens
    text = word_tokenize(text)
    return text

def filteringText(text): # Remove stopwors in a text
    listStopwords = set(stopwords.words('indonesian'))
    filtered = []
    for txt in text:
        if txt not in listStopwords:
            filtered.append(txt)
    text = filtered
    return text

def stemmingText(text): # Reducing a word to its word stem that affixes to suffixes and prefixes or to the roots of words
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    text = [stemmer.stem(word) for word in text]
    return text

def toSentence(list_words): # Convert list of words into sentence
    sentence = ' '.join(word for word in list_words)
    return sentence

##Download Punkt
Punkt dan stopword adalah corpus atau dataset yang disediakan oleh nltk yang akan banyak digunakan pada domain NLP

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

##Proses Preprocessing,
Tahap ini adalah menjalankan fungsi Preprocessing data kita

sebagai gambaran 4000 data sekitar 1.5 jam sabar ya

In [ ]:
tweets['text_clean'] = tweets['isitweet'].apply(cleaningText)
tweets['text_clean'] = tweets['text_clean'].apply(casefoldingText)
tweets.drop(['isitweet'], axis = 1, inplace = True)

tweets['text_preprocessed'] = tweets['text_clean'].apply(tokenizingText)
tweets['text_preprocessed'] = tweets['text_preprocessed'].apply(filteringText)
tweets['text_preprocessed'] = tweets['text_preprocessed'].apply(stemmingText)

# drop duplicates/spams tweets
tweets.drop_duplicates(subset = 'text_clean', inplace = True)

## Menampilkan kalimat yang sudah di preprocessing

In [ ]:
tweets

## Labeling dengan lexicon
Pelabelan data adalah proses penambahan label ke data mentah untuk memberikan konteks atau makna pada data. Data berlabel kemudian digunakan untuk melatih model NLP untuk membuat prediksi atau memahami atau menghasilkan ucapan.

pelabelan kali ini menggunakan lexicon sebagai pelabelan secara otomatis. Dalam klasifikasi berbasis leksikon, dokumen diberi label dengan membandingkan jumlah kata yang muncul dari dua leksikon yang berlawanan, seperti sentimen positif dan negatif.

Sebelum menjalankan kode berikut silahkan download 2 file pendukung berikut:

[Negatif](https://drive.google.com/file/d/1UBLrxmfFTEg1t9rHw6v-M5pJVRDWuyFH/view?usp=sharing)

[Positif](https://drive.google.com/file/d/1JqpJ12JqiOMOJ5R0Gg4rz3Xbqe3j2-Om/view?usp=sharing)

[Referensi](https://github.com/fajri91/InSet)

In [ ]:
# Loads lexicon positive and negative data
lexicon_positive = dict()
import csv
with open('positive.csv', 'r') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        lexicon_positive[row[0]] = int(row[1])

lexicon_negative = dict()
import csv
with open('negative.csv', 'r') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        lexicon_negative[row[0]] = int(row[1])

# Function to determine sentiment polarity of tweets
def sentiment_analysis_lexicon_indonesia(text):
    #for word in text:
    score = 0
    for word in text:
        if (word in lexicon_positive):
            score = score + lexicon_positive[word]
    for word in text:
        if (word in lexicon_negative):
            score = score + lexicon_negative[word]
    polarity=''
    if (score > 0):
        polarity = 'positive'
    elif (score < 0):
        polarity = 'negative'
    else:
        polarity = 'neutral'
    return score, polarity

In [ ]:
results = tweets['text_preprocessed'].apply(sentiment_analysis_lexicon_indonesia)
results = list(zip(*results))
tweets['polarity_score'] = results[0]
tweets['polarity'] = results[1]
print(tweets['polarity'].value_counts())

# Export to csv file


## Visualisasi Hasil sentiment

In [ ]:
fig, ax = plt.subplots(figsize = (6, 6))
sizes = [count for count in tweets['polarity'].value_counts()]
labels = list(tweets['polarity'].value_counts().index)
explode = (0.1, 0, 0)
ax.pie(x = sizes, labels = labels, autopct = '%1.1f%%', explode = explode, textprops={'fontsize': 14})
ax.set_title(f'Sentiment Polarity on Tweets Data \n (total = {len(tweets.index)} tweets)', fontsize = 16, pad = 20)
plt.show()

In [ ]:
pd.set_option('display.max_colwidth', 3000)
positive_tweets = tweets[tweets['polarity'] == 'positive']
positive_tweets = positive_tweets[['text_clean', 'polarity_score', 'polarity']].sort_values(by = 'polarity_score', ascending=False).reset_index(drop = True)
positive_tweets.index += 1
positive_tweets[0:10]

In [ ]:
pd.set_option('display.max_colwidth', 3000)
negative_tweets = tweets[tweets['polarity'] == 'negative']
negative_tweets = negative_tweets[['text_clean', 'polarity_score', 'polarity']].sort_values(by = 'polarity_score', ascending=True)[0:10].reset_index(drop = True)
negative_tweets.index += 1
negative_tweets[0:20]

##Visualize word cloud
Word cloud merupakan salah satu metode untuk menampilkan data teks secara visual. Grafik ini populer dalam text mining karena mudah dipahami.

In [ ]:
# Visualize word cloud

list_words=''
for tweet in tweets['text_preprocessed']:
    for word in tweet:
        list_words += ' '+(word)

wordcloud = WordCloud(width = 600, height = 400, background_color = 'black', min_font_size = 10).generate(list_words)
fig, ax = plt.subplots(figsize = (8, 6))
ax.set_title('Word Cloud of Tweets Data', fontsize = 18)
ax.grid(False)
ax.imshow((wordcloud))
fig.tight_layout(pad=0)
ax.axis('off')
plt.show()

#Tugas
Silahkan kalian labeling data yang sudah di crawling pada hari ke 2 kemarin sebagai mini project!

kemudian simpan hasilnya dalam csv!

In [ ]:
import pandas as pd

# Define the recipe data
data = {
    "name": ["Nasi Liwet", "Karedok", "Lotek"],
    "description": [
        "Nasi liwet adalah nasi gurih khas Sunda yang dimasak dengan santan dan rempah, sering disajikan dengan ikan asin, ayam, dan sayuran.",
        "Karedok adalah salad sayuran khas Sunda dengan bumbu kacang yang segar dan pedas.",
        "Lotek adalah hidangan khas Sunda yang mirip dengan gado-gado, terdiri dari sayuran rebus yang disajikan dengan bumbu kacang."
    ],
    "ingredients": [
        "300 gram beras, 400 ml air, 200 ml santan, 1 batang serai, 2 lembar daun salam, 2 lembar daun jeruk, 1 sendok teh garam",
        "100 gram kacang panjang, 100 gram tauge, 50 gram kol, 1 buah mentimun, 5 lembar daun kemangi",
        "100 gram kacang panjang, 100 gram tauge, 50 gram kol, 100 gram bayam, 1 buah ketimun"
    ],
    "steps": [
        "Cuci beras hingga bersih. Masukkan beras, air, santan, serai, daun salam, daun jeruk, dan garam ke dalam rice cooker atau panci. Masak hingga nasi matang dan bumbu meresap. Setelah matang, angkat dan sajikan nasi liwet dengan lauk pendamping.",
        "Siapkan sayuran, cuci bersih dan tiriskan. Haluskan bumbu kacang, lalu campur dengan sayuran. Aduk rata dan sajikan.",
        "Rebus semua sayuran hingga setengah matang, lalu tiriskan. Haluskan bumbu kacang, campurkan dengan sayuran rebus. Sajikan lotek dengan tambahan lontong atau ketupat jika diinginkan."
    ],
    "bumbu_kacang": [
        None,
        "100 gram kacang tanah, 3 siung bawang putih, 5 buah cabai rawit merah, 2 cm kencur, 1 sendok teh gula merah, 1 sendok teh garam, 100 ml air matang",
        "100 gram kacang tanah, 3 siung bawang putih, 5 buah cabai merah, 1 sendok teh gula merah, 1 sendok teh garam, 100 ml air matang"
    ]
}

# Create DataFrame
df = pd.DataFrame(data)

# Save the DataFrame as CSV
df.to_csv('recipe_data.csv', index=False)
print("DataFrame saved to recipe_data.csv")